In [2]:
import pandas as pd

orders = pd.read_csv("orders.csv")
orders.head()
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [3]:
import sqlite3

# Create database
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

# Execute SQL file
with open("restaurants.sql", "r") as file:
    cursor.executescript(file.read())

# Load restaurant table
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [4]:
merged_1 = orders.merge(
    users,
    on="user_id",
    how="left"
)


In [5]:
final_df = merged_1.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)


In [10]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [11]:
gold_df = final_df[final_df["membership"] == "Gold"]
result = gold_df.groupby("city")["total_amount"].sum().sort_values(ascending=False)

result


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [13]:
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [14]:
user_totals = final_df.groupby("user_id")["total_amount"].sum()
(user_totals > 1000).sum()


np.int64(2544)

In [16]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

final_df["rating_range"] = pd.cut(final_df["rating"], bins=bins, labels=labels)
final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)



/tmp/ipython-input-1814876920.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6–5.0,2197030.75
4.1–4.5,1960326.26
3.0–3.5,1881754.57
3.6–4.0,1717494.41


In [17]:
gold_df = final_df[final_df["membership"] == "Gold"]

gold_df.groupby("city")["total_amount"].mean().sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [18]:
final_df.groupby("cuisine").agg(
    distinct_restaurants=("restaurant_id", "nunique"),
    total_revenue=("total_amount", "sum")
).sort_values("distinct_restaurants")


,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [19]:
total_orders = len(final_df)
gold_orders = len(final_df[final_df["membership"] == "Gold"])

round((gold_orders / total_orders) * 100)


50

In [21]:
restaurant_stats = final_df.groupby("restaurant_name_x").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

restaurant_stats[restaurant_stats["total_orders"] < 20] \
    .sort_values("avg_order_value", ascending=False)


,avg_order_value,total_orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19
...,...,...
Annapurna Tiffins Punjabi,621.828947,19
Darbar Tiffins Non-Veg,596.815556,18
Darbar Restaurant Punjabi,589.972857,14


In [22]:
final_df.groupby(["membership", "cuisine"])["total_amount"].sum() \
  .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [23]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")


/tmp/ipython-input-2008092552.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


In [25]:
final_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [26]:
final_df = pd.read_csv("final_food_delivery_dataset.csv")
len(final_df[final_df["membership"] == "Gold"])


4987

In [27]:
round(final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum())


1889367

In [28]:
final_df["user_id"].nunique()


2883

In [29]:
round(final_df[final_df["membership"] == "Gold"]["total_amount"].mean(), 2)


np.float64(797.15)

In [30]:
len(final_df[final_df["rating"] >= 4.5])


3374

In [31]:
gold_city_revenue = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
)

gold_city_revenue


,total_amount
city,
Bangalore,994702.59
Chennai,1080909.79
Hyderabad,896740.19
Pune,1003012.32


In [32]:
len(
    final_df[
        (final_df["membership"] == "Gold") &
        (final_df["city"] == "Chennai")
    ]
)


1337